In [1]:
import requests, json, os
from tqdm import tqdm

In [2]:
s = requests.Session()
s.headers.update({'User-Agent': 'ArcGIS'})
response = s.get("https://broadbandmap.fcc.gov/nbm/map/api/published/filing")
parsed = json.loads(response.text)
uuid = parsed['data'][0]['process_uuid']
url = f'https://broadbandmap.fcc.gov/nbm/map/api/national_map_process/nbm_get_data_download/{uuid}'
response = s.get(url)
parsed = json.loads(response.text)

In [3]:
stateList = list(set([item['state_name'] for item in parsed['data']]))
stateList.remove(None)
stateList.sort()
stateList = ['All'] + stateList

# statesToProcess = ['California', 'Nevada', 'Washington', 'Idaho', 'Oregon', 'Pennsylvania']
statesToProcess = 'All'

In [4]:
dataToProcess = parsed['data']
dataToProcess = [item for item in dataToProcess if item['state_name'] != None]

if statesToProcess != 'All':
    dataToProcess = [item for item in dataToProcess if item['state_name'] in statesToProcess]

dataToProcess = [item for item in dataToProcess if item['file_type'] == 'csv']

In [5]:
if not os.path.isdir('data'):
    os.makedirs('data')

for item in tqdm(dataToProcess):
    # print(item)
    url = f"https://broadbandmap.fcc.gov/nbm/map/api/getNBMDataDownloadFile/{item['id']}/1"
    state = item["file_name"].split('_')[1]
    r = s.get(url)
    if not os.path.isdir(f'data\{state}'):
        os.makedirs(f'data\{state}')
    open(f'data\\{state}\\{item["file_name"]}.zip', 'wb').write(r.content)

100%|██████████| 389/389 [13:34<00:00,  2.09s/it]
